In [1]:
import tensorflow
tensorflow.keras.__version__

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
import pandas as pd
df = pd.read_csv("HR_Mockdata.csv")
df = df.drop(columns=["sales \nfunction", "hire_date", 'move_date', 'end_date',
                    'country_home', 'country_host', 'business Size_host', 
                      'economic Perspective_host', 'left', 'fired_quit'])
# Drop null columns/rows
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.tail()

,#,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,leader_host,culture_home,culture_host,function_host,buzz_bust
14994,10244,1.0,0.63,2,105,2,0,0,medium,key talent,consensual,egalitarian,operations,buzz
14995,10326,1.0,0.89,4,254,3,1,0,medium,well placed talent,hierarquical,egalitarian,operations,buzz
14996,10918,1.0,0.73,4,240,4,0,0,low,well placed talent,hierarquical,egalitarian,operations,bust
14997,11376,1.0,0.93,5,167,3,0,0,medium,well placed talent,hierarquical,egalitarian,operations,buzz
14998,12828,1.0,0.55,3,170,3,0,0,medium,well placed talent,consensual,egalitarian,regulatory affairs,buzz


In [3]:
list(df.columns.values)

['#',
 'satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'function_host',
 'buzz_bust']

In [4]:
# select column names that need to be converted to dummy variables
to_dummify = [
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'function_host'
 ]

In [5]:
for dummy in to_dummify:
    df = pd.concat([df.drop(dummy, axis=1), pd.get_dummies(df[dummy], 
                prefix=dummy)], axis=1)
df.head()

,#,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,buzz_bust,salary_high,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,45,0.09,0.89,6,276,4,0,0,bust,0,...,0,0,0,0,0,0,1,0,0,0
1,513,0.09,0.96,6,259,4,0,0,buzz,0,...,0,0,0,0,0,0,0,0,0,0
2,529,0.09,0.80,6,301,5,0,0,buzz,0,...,0,0,0,0,0,0,0,0,0,0
3,689,0.09,0.82,6,257,4,0,0,buzz,0,...,0,0,0,0,0,0,0,0,0,0
4,1189,0.09,0.80,5,279,4,0,0,buzz,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df.drop("buzz_bust", axis=1)
y = df["buzz_bust"]
print(X.shape, y.shape)
X.head(1)

(14999, 35) (14999,)


,#,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_high,salary_low,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,45,0.09,0.89,6,276,4,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    random_state=1, stratify=y)

In [8]:
from sklearn.preprocessing import MinMaxScaler
# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

C:\Users\h4wth\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [9]:
# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [15]:

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=35))
model.add(Dense(units=2, activation='softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 216       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 14        
Total params: 230
Trainable params: 230
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 2s - loss: 0.5693 - acc: 0.7441
Epoch 2/100
 - 1s - loss: 0.5365 - acc: 0.7678
Epoch 3/100
 - 1s - loss: 0.5327 - acc: 0.7678
Epoch 4/100
 - 1s - loss: 0.5300 - acc: 0.7678
Epoch 5/100
 - 1s - loss: 0.5272 - acc: 0.7678
Epoch 6/100
 - 1s - loss: 0.5242 - acc: 0.7678
Epoch 7/100
 - 1s - loss: 0.5209 - acc: 0.7678
Epoch 8/100
 - 1s - loss: 0.5176 - acc: 0.7679
Epoch 9/100
 - 1s - loss: 0.5147 - acc: 0.7678
Epoch 10/100
 - 1s - loss: 0.5123 - acc: 0.7680
Epoch 11/100
 - 1s - loss: 0.5102 - acc: 0.7677
Epoch 12/100
 - 1s - loss: 0.5085 - acc: 0.7674
Epoch 13/100
 - 1s - loss: 0.5068 - acc: 0.7674
Epoch 14/100
 - 1s - loss: 0.5055 - acc: 0.7678
Epoch 15/100
 - 1s - loss: 0.5040 - acc: 0.7674
Epoch 16/100
 - 1s - loss: 0.5030 - acc: 0.7680
Epoch 17/100
 - 1s - loss: 0.5018 - acc: 0.7677
Epoch 18/100
 - 1s - loss: 0.5011 - acc: 0.7676
Epoch 19/100
 - 1s - loss: 0.5003 - acc: 0.7674
Epoch 20/100
 - 1s - loss: 0.4993 - acc: 0.7681
E

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.4941 - acc: 0.7712
Normal Neural Network - Loss: 0.494071794907252, Accuracy: 0.7712000012397766
